In [31]:
import geopandas as gpd
import pandas as pd
import itertools

In [32]:
if "snakemake" in locals():
    input_path = snakemake.input[0]
    output_path = snakemake.output[1]

else:
    input_path = "../../resources/paris/spatial.gpkg"
    output_path = None

In [33]:
# Load areas
df = gpd.read_file(input_path)

In [ ]:
# Keep network
df_network = df[df["name"] == "network"].copy()
assert len(df_network) == 1

df = df[df["name"] != "network"].copy()

In [35]:
# Construct all combinations
names = df["name"].unique()
cases = []

for k in range(1, len(names) + 1):
    cases += itertools.combinations(names, k)

if "main" in names:
    cases = [case for case in cases if "main" in case]

In [36]:
df_result = [df_network]

# Construct combined geometries
for case in cases:
    df_case = df[df["name"].isin(case)].dissolve()
    df_case["name"] = "+".join(case)
    df_result.append(df_case)

df_result = pd.concat(df_result)

In [ ]:
# Output
df_result.to_file(output_path)